In [2]:
import pandas as pd
import numpy as np
import io
import os
import json
import distutils.dir_util
from collections import Counter
from tensorflow.keras.models import save_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense, SimpleRNN,Dropout

## 데이터 전처리

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

Mounted at /gdrive


In [12]:
# train data 준비
f_train = pd.read_json('./kakao_arena_melon/arena_data/orig/train.json' ,typ = 'frame', encoding='utf-8')
df_train = pd.DataFrame(f_train)
df_train = df_train.sort_values(by=['like_cnt'],ascending=False)
df_train = df_train[df_train['like_cnt']>15]
df_train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
8699,"[노동요, 취향저격, 드라이브, Pop, 기분전환, 트렌디, 스트레스, 신나는]",74220,HOT TRENDY POP: 놓쳐선 안될 'POP' (매주 업데이트),"[159317, 368264, 600999, 274905, 643023, 24070...",53211,2020-04-17 14:32:23.000
61103,"[감성, 새벽, 때껄룩, 잔잔한, 휴식, 팝송, 떼껄룩, 힐링, 팝, 신나는]",122388,❤️때껄룩님 ᴛᴀᴋᴇ ᴀ ʟᴏᴏᴋ 플레이리스트❤️,"[532771, 383775, 432753, 176384, 152694, 60126...",41844,2020-04-23 20:58:45.000
19863,"[매장음악, 카페, 러블리쏘, 분위기, 드라이브, 여행, 매력적인, 기분전환, 팝]",26174,듣다보면 '우와!' 하고 제목을 보게되는 팝,"[40160, 20039, 561488, 20051, 16242, 256568, 7...",27268,2020-04-19 21:22:11.000


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30364 entries, 8699 to 6203
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          30364 non-null  object
 1   id            30364 non-null  int64 
 2   plylst_title  30364 non-null  object
 3   songs         30364 non-null  object
 4   like_cnt      30364 non-null  int64 
 5   updt_date     30364 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.6+ MB


In [14]:
# validation data 준비
f_val = pd.read_json('./kakao_arena_melon/arena_data/orig/val.json' ,typ = 'frame', encoding="utf-8")
df_val = pd.DataFrame(f_val)
df_val = df_val.sort_values(by=['like_cnt'],ascending=False)
df_val = df_val[df_val['like_cnt']>15]
df_val.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
16571,"[봄, 가을, 달달, 명곡, 힐링, 설렘, 사랑]",102123,여자들 취향저격하는 달달한 노래,"[481910, 216728, 460884, 547506, 541420, 10636...",20149,2019-11-08 05:31:14.000
20334,"[감성, 밤, 여름밤, 새벽감성, 새벽, 밤에듣기좋은노래, 새벽에듣기좋은노래, 잠들...",11762,불 꺼진 방 침대 속 자기 직전... 듣기 좋은 노래 ⭐,"[13815, 366786, 454528, 451593, 61159, 231078,...",19758,2020-04-17 03:16:32.000
2587,[팝],56212,내가 뽑은 최고의 팝송 TOP 100,"[672232, 430846, 264908, 588165, 19486, 668733...",18403,2012-03-19 10:49:17.000
21280,"[팝, 잔잔한, 매장음악, 까페, 편안한]",49159,편하게 듣는 사랑받는 팝 BEST,"[287098, 95310, 175230, 70704, 705515, 495979,...",16605,2016-08-15 18:31:56.000
19899,"[집중, 연주곡, 잔잔한, 피아노, 수험생, 일할때, 뉴에이지, 공부, 북카페]",46039,공부할때 마다 듣는 잔잔한 피아노,"[552555, 518831, 643070, 418165, 644026, 32046...",14371,2018-05-04 21:18:04.000


In [15]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7396 entries, 16571 to 12748
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          7396 non-null   object
 1   id            7396 non-null   int64 
 2   plylst_title  7396 non-null   object
 3   songs         7396 non-null   object
 4   like_cnt      7396 non-null   int64 
 5   updt_date     7396 non-null   object
dtypes: int64(2), object(4)
memory usage: 404.5+ KB


In [16]:
# 플레이리스트 내 song id 리스트
train_song = df_train['songs']
# 플레이리스트 내 tag 리스트
train_tag = df_train['tags']
# 플레이리스트 내 song id 리스트 전체 나열
song_list = [song for plist in train_song for song in plist]
# 플레이리스트 내 tag 리스트 전체 나열
tag_list = [tag for plist in train_tag for tag in plist]

In [17]:
# 전체 나열 리스트 중 각 song의 개수
count_song = Counter(song_list)
# 전체 나열 리스트 중 각 tag의 개수
count_tag = Counter(tag_list)

In [18]:
# 전체 플레이리스트 내 20회 넘게 담긴 song을 dictionary에 추가
mfl={}  # meaningful
for key, value in count_song.items():
    if value>20:
        mfl[key]=value
song_len = len(mfl) # 20회 이상 담긴 song의 개수
song_len

16019

In [19]:
# 전체 플레이리스트 내 10회 넘게 담긴 tag을 dictionary에 추가
for key, value in count_tag.items():
    if value>10:
        mfl[key]=value
tag_len=len(mfl)-song_len # 11회 이상 담긴 tag의 개수
tag_len

1160

In [20]:
mfl_col = list(mfl.keys())
len(mfl_col) # 유의미한 song과 tag의 총 개수 (song_len+tag_len)

17179

In [21]:
# 전체 플레이리스트를 row, 유의미한 song과 tag를 column으로 하는 zero matrix 생성
zero_matrix = np.zeros((len(df_train),len(mfl_col)))
zero_matrix.shape

(30364, 17179)

In [22]:
df_zero = pd.DataFrame(zero_matrix,columns=mfl_col,index=df_train['id'])
df_zero.head() # matrix >> dataframe 변환

,643023,424328,380410,443160,532771,383775,432753,152694,330285,142570,...,빌로우,금요일,타이틀곡,tv별책부록,부드러운_잔잔한,세븐틴,RandB,코노,휠링,러블리즈
id,,,,,,,,,,,,,,,,,,,,,
74220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# 각 플레이리스트에 해당하는 곡, 태그 좌표에 1 부여

def create_onehot(df,column_name):
    zero_matrix=np.zeros((len(df),len(column_name)))
    zero_df=pd.DataFrame(zero_matrix,columns=column_name,index=df['id'])
    for i in range(len(df)):
        for tag,song in zip(df.iloc[i,0],df.iloc[i,3]):
            if tag in column_name:
                zero_df.iloc[i,column_name.index(tag)]=1
            if song in column_name:
                zero_df.iloc[i,column_name.index(song)]=1
    return zero_df

In [25]:
# create_onehot 함수 활용 train data >> onehot encoding
train_onehot = create_onehot(df_train, mfl_col)
train_onehot.shape

(30364, 17179)

In [26]:
# create_onehot 함수 활용 validation data >> onehot encoding
val_onehot = create_onehot(df_val, mfl_col)
val_onehot.shape

(7396, 17179)

## 오토인코더(학습)

In [27]:
encoding_dim=64
input_pl = Input(shape=(len(mfl_col),)) # one-hot encoding된 DataFrame의 행이 Input
dropout = Dropout(0.2)(input_pl)
encoded = Dense(encoding_dim,activation='relu')(input_pl) # 64개로 축소(인코딩)
encoded = Dense(36,activation='relu')(encoded) # 36개로 축소(인코딩)

decoded = Dense(len(mfl_col),activation='sigmoid')(encoded) # 다시 원래 column 개수로 복원(디코딩)
autoencoder = Model(input_pl,decoded)

autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 17179)]           0         
                                                                 
 dense (Dense)               (None, 64)                1099520   
                                                                 
 dense_1 (Dense)             (None, 36)                2340      
                                                                 
 dense_2 (Dense)             (None, 17179)             635623    
                                                                 
Total params: 1737483 (6.63 MB)
Trainable params: 1737483 (6.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
autoencoder.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
history = autoencoder.fit(train_onehot,train_onehot,epochs=5,batch_size=64,validation_split=0.2,verbose=1) # 기존의 데이터프레임 복원이 목적이기 때문에, test data == train data

Epoch 1/5
380/380 [==============================] - 26s 67ms/step - loss: 0.1006 - accuracy: 0.0166 - val_loss: 0.0037 - val_accuracy: 0.0171
Epoch 2/5
380/380 [==============================] - 26s 69ms/step - loss: 0.0034 - accuracy: 0.0204 - val_loss: 0.0031 - val_accuracy: 0.0171
Epoch 3/5
380/380 [==============================] - 26s 69ms/step - loss: 0.0029 - accuracy: 0.0205 - val_loss: 0.0028 - val_accuracy: 0.0171
Epoch 4/5
380/380 [==============================] - 26s 69ms/step - loss: 0.0026 - accuracy: 0.0204 - val_loss: 0.0026 - val_accuracy: 0.0171
Epoch 5/5
380/380 [==============================] - 26s 68ms/step - loss: 0.0025 - accuracy: 0.0205 - val_loss: 0.0025 - val_accuracy: 0.0181


In [30]:
history = autoencoder.fit(train_onehot,train_onehot,epochs=5,batch_size=64,validation_split=0.2,verbose=1) # 기존의 데이터프레임 복원이 목적이기 때문에, test data == train data

Epoch 1/5
380/380 [==============================] - 27s 70ms/step - loss: 0.0024 - accuracy: 0.0277 - val_loss: 0.0024 - val_accuracy: 0.0179
Epoch 2/5
380/380 [==============================] - 26s 70ms/step - loss: 0.0024 - accuracy: 0.0310 - val_loss: 0.0023 - val_accuracy: 0.0171
Epoch 3/5
380/380 [==============================] - 27s 72ms/step - loss: 0.0023 - accuracy: 0.0331 - val_loss: 0.0023 - val_accuracy: 0.0478
Epoch 4/5
380/380 [==============================] - 26s 69ms/step - loss: 0.0023 - accuracy: 0.0361 - val_loss: 0.0023 - val_accuracy: 0.0336
Epoch 5/5
380/380 [==============================] - 26s 67ms/step - loss: 0.0023 - accuracy: 0.0383 - val_loss: 0.0022 - val_accuracy: 0.0403


In [31]:
predict_plist = autoencoder.predict(val_onehot)

232/232 [==============================] - 2s 7ms/step


In [32]:
df_id = list(df_val['id'])
col= mfl_col
ori_song = col[:song_len]
ori_tag = col[song_len:]

song_predict = predict_plist[:,:song_len] # song output(추천곡)
tag_predict = predict_plist[:,song_len:] # tag output(추천태그)

In [33]:
result=[]
n=0
for i in df_id:
    dic={}
    dic['id']=i

    plist_song=song_predict[n].argsort()[-100:] # predict한 song output 중 상위 100개
    p_song=[]
    for song in plist_song:
        p_song.append(ori_song[song])
    dic['songs']=p_song

    plist_tag=tag_predict[n].argsort()[-10:] # predict한 tag output 중 상위 10개
    p_tag=[]
    for tag in plist_tag:
        p_tag.append(ori_tag[tag])
    dic['tags']=p_tag
    n+=1
    result.append(dic)

In [34]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname+".json")
    distutils.dir_util.mkpath(parent)
    with io.open( fname+".json", "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [35]:
write_json(result,'results')

In [49]:
a = pd.read_json('./kakao_arena_melon/arena_data/answers/val.json' ,typ = 'frame', encoding='utf-8')
a = a.sort_values(by=['like_cnt'],ascending=False)
a = a[a['like_cnt']>15]
a.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
14820,"[봄, 가을, 달달, 사랑]",102123,여자들 취향저격하는 달달한 노래,"[481910, 460884, 226792, 389446, 194190, 60151...",20149,2019-11-08 05:31:14.000
14743,"[새벽감성, 새벽에듣기좋은노래, 잠들기전, 공부, 인디]",11762,불 꺼진 방 침대 속 자기 직전... 듣기 좋은 노래 ⭐,"[13815, 366786, 454528, 451593, 61159, 231078,...",19758,2020-04-17 03:16:32.000
796,[팝],56212,내가 뽑은 최고의 팝송 TOP 100,"[430846, 264908, 588165, 284554, 669677, 59771...",18403,2012-03-19 10:49:17.000
18272,"[잔잔한, 매장음악, 편안한]",49159,편하게 듣는 사랑받는 팝 BEST,"[287098, 95310, 175230, 70704, 705515, 495979,...",16605,2016-08-15 18:31:56.000
6575,"[연주곡, 피아노, 수험생, 공부, 북카페]",46039,공부할때 마다 듣는 잔잔한 피아노,"[552555, 518831, 643070, 418165, 644026, 32046...",14371,2018-05-04 21:18:04.000


In [46]:
a.shape

(7396, 6)

In [50]:
a.to_json('answers.json', force_ascii=False, orient = 'records')

In [41]:
b = pd.read_json('results.json' ,typ = 'frame', encoding='utf-8')
b.shape

(7396, 3)